In [2]:
import folium
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import io, base64
from folium import raster_layers
from branca.colormap import LinearColormap

In [4]:
def create_raster_overlay(adf_path):
    with rasterio.open(adf_path) as src:
        grid = src.read(1)
        bounds = [[src.bounds.bottom, src.bounds.left], [src.bounds.top, src.bounds.right]]

    grid[grid == -9999] = np.nan
    grid[grid < 0] = np.nan
    valid = np.nan_to_num(grid, nan=0)
    vmin, vmax = valid[valid > 0].min(), valid.max()

    # Create PNG
    fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
    masked = np.ma.masked_where(valid == 0, valid)
    ax.imshow(masked, cmap='viridis', vmin=vmin, vmax=vmax)
    ax.axis('off')
    buf = io.BytesIO()
    fig.savefig(buf, format='png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    png_uri = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode()

    # Create legend
    colors = [plt.cm.viridis(i) for i in np.linspace(0, 1, 10)]
    hex_colors = [mcolors.rgb2hex(c) for c in colors]
    legend = LinearColormap(hex_colors, vmin=vmin, vmax=vmax, caption="Rainfall (mm)")

    return png_uri, bounds, legend

def add_vector_layer(map_obj, gdf, name, style_func, tooltip_fields=None, tooltip_aliases=None):
    tooltip = None
    if tooltip_fields:
        tooltip = folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
    folium.GeoJson(gdf, name=name, style_function=style_func, tooltip=tooltip).add_to(map_obj)

In [10]:
ohia = gpd.read_file("data/metpoly/metpoly.shp").set_crs(epsg=32604, allow_override=True).to_crs(epsg=4326)

# Elevation contours
contours = gpd.read_file("data/Hawaii_Elevation_Contours_100ft").to_crs(epsg=4326)

# Precipitation raster (convert to PNG)
adf_path = "data/StateRFGrids_mm/staterf_mm01/w001001x.adf"
rain_png, rain_bounds, rain_legend = create_raster_overlay(adf_path)

In [12]:
m = folium.Map(location=[19.7, -155.5], zoom_start=8, tiles="CartoDB positron")

# Add precipitation image overlay
raster_layers.ImageOverlay(
    image=rain_png,
    bounds=rain_bounds,
    opacity=0.6,
    name="Precipitation (Nov, mm)"
).add_to(m)

# Add ʻŌhiʻa range polygon layer
add_vector_layer(
    m,
    ohia,
    name="ʻŌhiʻa lehua range",
    style_func=lambda feat: {
        "fillColor": "#dd0000",
        "color": "#990000",
        "weight": 1,
        "fillOpacity": 0.2
    },
    tooltip_fields=["ID", "GRIDCODE"],
    tooltip_aliases=["ID", "Class"]
)

# Add elevation contours
add_vector_layer(
    m,
    contours,
    name="Elevation Contours",
    style_func=lambda feat: {
        "color": "gray",
        "weight": 0.7,
        "dashArray": "3, 3"
    },
    tooltip_fields=["contour"],
    tooltip_aliases=["Elevation (ft)"]
)

# Add legend and controls
rain_legend.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# Save the map
m.save("hawaii_combined_map.html")